# Pandas-2 (NaN, GroupBy/Pivot, Merge/Join)
Bu notebook, Pandas'ın şu konularını verir:

- Eksik veri (NaN) bulma ve düzeltme
- GroupBy ile özet tablolar
- Pivot table ile "rapor" çıkarma
- Merge/Join ile tablolar birleştirme
- Basit veri temizleme (duplicate, dtype)


## 1) Kurulum


In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
print("pandas surumu:", pd.__version__)


pandas surumu: 2.2.2


## 2) Mini Örnek


### 2.1) İki tablo üretelim: öğrenciler ve bölümler


In [ ]:
ogrenciler = pd.DataFrame({
    "ogrenci_id": [1, 2, 3, 4],
    "ad": ["Ali", "Ayse", "Mehmet", "Elif"],
    "bolum": ["YBS", "YBS", "Isletme", "YBS"],
    "gpa": [3.1, np.nan, 2.4, 3.7]
})

bolumler = pd.DataFrame({
    "bolum": ["YBS", "Isletme"],
    "fakulte": ["IIBF", "IIBF"]
})

ogrenciler, bolumler


(   ogrenci_id      ad    bolum  gpa
 0           1     Ali      YBS  3.1
 1           2    Ayse      YBS  NaN
 2           3  Mehmet  Isletme  2.4
 3           4    Elif      YBS  3.7,
      bolum fakulte
 0      YBS    IIBF
 1  Isletme    IIBF)

### 2.2) NaN bulma ve doldurma


In [ ]:
ogrenciler.isna()


,ogrenci_id,ad,bolum,gpa
0,False,False,False,False
1,False,False,False,True
2,False,False,False,False
3,False,False,False,False


In [ ]:
ogrenciler["gpa"].fillna(ogrenciler["gpa"].mean(), inplace=True)
ogrenciler


/tmp/ipython-input-322095952.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ogrenciler["gpa"].fillna(ogrenciler["gpa"].mean(), inplace=True)


,ogrenci_id,ad,bolum,gpa
0,1,Ali,YBS,3.100000
1,2,Ayse,YBS,3.066667
2,3,Mehmet,Isletme,2.400000
3,4,Elif,YBS,3.700000


### 2.3) Merge (Tabloları birleştirme)


In [ ]:
df_mini = ogrenciler.merge(bolumler, on="bolum", how="left")
df_mini


,ogrenci_id,ad,bolum,gpa,fakulte
0,1,Ali,YBS,3.100000,IIBF
1,2,Ayse,YBS,3.066667,IIBF
2,3,Mehmet,Isletme,2.400000,IIBF
3,4,Elif,YBS,3.700000,IIBF


### 2.4) GroupBy ile özet


In [ ]:
df_mini.groupby("bolum")["gpa"].agg(["count","mean"]).reset_index()


,bolum,count,mean
0,Isletme,1,2.400000
1,YBS,3,3.288889


---
# 3. Bölüm — Sentetik CSV ile Gerçekçi Çalışma
Bu kisimda 3 ayri tablo kullanacagiz:
- öğrenciler (ybs_ogrenciler_ascii.csv)
- dersler    (ybs_dersler_ascii.csv)
- notlar     (ybs_notlar_ascii.csv)

Not: Dosyalar aynı notebook dosyasında olmalı.


## 4) CSV okuma


In [ ]:
ogr = pd.read_csv("ybs_ogrenciler_ascii.csv")
ders = pd.read_csv("ybs_dersler_ascii.csv")
notlar = pd.read_csv("ybs_notlar_ascii.csv")

ogr.head()


,ogrenci_id,ad,soyad,sehir,bolum,sinif,gpa,email
0,200001,Burak,Celik,Samsun,YBS,4,1.76,burak.celik33@example.com
1,200002,Ece,Yildiz,Adana,Endustri,4,2.48,ece.yildiz54@example.com
2,200003,Mert,Karaca,Konya,Ekonomi,2,2.28,mert.karaca44@example.com
3,200004,Can,Tas,Samsun,YBS,3,3.13,can.tas93@example.com
4,200005,Sibel,Kurt,Kocaeli,YBS,2,2.98,sibel.kurt31@example.com


In [ ]:
ders.head()


,ders_kodu,ders_adi,kategori,kredi
0,DS101,VeriBilimi,Data,3
1,DB201,VeriTabani,Data,3
2,PY101,PythonProgramlama,Tech,2
3,ST201,Istatistik,Math,2
4,BA301,IsAnalitigi,Business,3


In [ ]:
notlar.head()


,kayit_id,ogrenci_id,ders_kodu,donem,vize,final
0,500001,200189,DS101,2026-Spring,84.8,62.7
1,500002,200160,AI201,2025-Fall,65.7,79.2
2,500003,200183,MK201,2025-Fall,100.0,51.0
3,500004,200202,FN201,2026-Spring,67.4,NaN
4,500005,200047,AI201,2025-Fall,62.6,97.5


## 5) Eksik veri (NaN) — çekirdek işlemler


### 5.1) NaN sayıları


In [ ]:
ogr.isna().sum()


,0
ogrenci_id,0
ad,0
soyad,0
sehir,0
bolum,0
sinif,0
gpa,15
email,16


In [ ]:
notlar.isna().sum()


,0
kayit_id,0
ogrenci_id,0
ders_kodu,0
donem,0
vize,55
final,68


### 5.2) Eksik gpa değerlerini doldurma (mean ile)


In [ ]:
ogr["gpa"] = ogr["gpa"].fillna(ogr["gpa"].mean())
ogr.isna().sum()


,0
ogrenci_id,0
ad,0
soyad,0
sehir,0
bolum,0
sinif,0
gpa,0
email,16


### 5.3) Notlarda eksik vize/final var: satır atmak mı, doldurmak mı?


Eksik olanları 0 ile dolduralım (sınava girmedi gibi).


In [ ]:
notlar[["vize","final"]] = notlar[["vize","final"]].fillna(0)
notlar.isna().sum()


,0
kayit_id,0
ogrenci_id,0
ders_kodu,0
donem,0
vize,0
final,0


## 6) Yeni kolon: toplam_not ve gecti_kaldi


Basit bir not formulu: toplam = 0.4 * vize + 0.6 * final


In [ ]:
notlar["toplam_not"] = 0.4*notlar["vize"] + 0.6*notlar["final"]
notlar["gecti_kaldi"] = np.where(notlar["toplam_not"] >= 60, "Gecti", "Kaldi")
notlar.head()


,kayit_id,ogrenci_id,ders_kodu,donem,vize,final,toplam_not,gecti_kaldi
0,500001,200189,DS101,2026-Spring,84.8,62.7,71.54,Gecti
1,500002,200160,AI201,2025-Fall,65.7,79.2,73.80,Gecti
2,500003,200183,MK201,2025-Fall,100.0,51.0,70.60,Gecti
3,500004,200202,FN201,2026-Spring,67.4,0.0,26.96,Kaldi
4,500005,200047,AI201,2025-Fall,62.6,97.5,83.54,Gecti


## 7) Merge/Join — 3 tabloyu birleştirme


### 7.1) Notlar + öğrenciler


In [ ]:
df = notlar.merge(ogr, on="ogrenci_id", how="left")
df.head()


,kayit_id,ogrenci_id,ders_kodu,donem,vize,final,toplam_not,gecti_kaldi,ad,soyad,sehir,bolum,sinif,gpa,email
0,500001,200189,DS101,2026-Spring,84.8,62.7,71.54,Gecti,Ece,Gunes,Istanbul,Isletme,3,3.270000,ece.gunes59@example.com
1,500002,200160,AI201,2025-Fall,65.7,79.2,73.80,Gecti,Sena,Ozdemir,Eskisehir,Isletme,4,1.970000,NaN
2,500003,200183,MK201,2025-Fall,100.0,51.0,70.60,Gecti,Kerem,Arslan,Bursa,Bilgisayar,4,2.860000,kerem.arslan41@example.com
3,500004,200202,FN201,2026-Spring,67.4,0.0,26.96,Kaldi,Ozan,Yildiz,Eskisehir,Endustri,4,2.490000,ozan.yildiz52@example.com
4,500005,200047,AI201,2025-Fall,62.6,97.5,83.54,Gecti,Sena,Aydin,Bursa,Isletme,2,2.690244,sena.aydin89@example.com


### 7.2) + ders bilgileri


In [ ]:
df = df.merge(ders, on="ders_kodu", how="left")
df.head()


,kayit_id,ogrenci_id,ders_kodu,donem,vize,final,toplam_not,gecti_kaldi,ad,soyad,sehir,bolum,sinif,gpa,email,ders_adi,kategori,kredi
0,500001,200189,DS101,2026-Spring,84.8,62.7,71.54,Gecti,Ece,Gunes,Istanbul,Isletme,3,3.270000,ece.gunes59@example.com,VeriBilimi,Data,3
1,500002,200160,AI201,2025-Fall,65.7,79.2,73.80,Gecti,Sena,Ozdemir,Eskisehir,Isletme,4,1.970000,NaN,YapayZekaTemelleri,Tech,3
2,500003,200183,MK201,2025-Fall,100.0,51.0,70.60,Gecti,Kerem,Arslan,Bursa,Bilgisayar,4,2.860000,kerem.arslan41@example.com,PazarlamaAnalizi,Business,4
3,500004,200202,FN201,2026-Spring,67.4,0.0,26.96,Kaldi,Ozan,Yildiz,Eskisehir,Endustri,4,2.490000,ozan.yildiz52@example.com,FinansAnalizi,Business,3
4,500005,200047,AI201,2025-Fall,62.6,97.5,83.54,Gecti,Sena,Aydin,Bursa,Isletme,2,2.690244,sena.aydin89@example.com,YapayZekaTemelleri,Tech,3


## 8) GroupBy — özet raporlar


### 8.1) Bölüm bazında başarı ortalaması


In [ ]:
df.groupby("bolum")["toplam_not"].agg(["count","mean","min","max"]).reset_index().sort_values("mean", ascending=False)


,bolum,count,mean,min,max
1,Ekonomi,56,64.812500,16.20,95.44
3,Isletme,207,64.480773,0.00,94.78
2,Endustri,177,64.414576,10.32,94.76
4,YBS,398,61.950553,7.14,99.16
0,Bilgisayar,62,61.854516,18.32,95.04


### 8.2) Ders bazında geçme oranı


In [ ]:
rapor_ders = df.groupby("ders_kodu").agg(
    kayit_sayisi=("kayit_id","count"),
    ortalama=("toplam_not","mean"),
    gecen_sayi=("gecti_kaldi", lambda x: (x=="Gecti").sum())
).reset_index()

rapor_ders["gecme_orani"] = rapor_ders["gecen_sayi"] / rapor_ders["kayit_sayisi"]
rapor_ders.sort_values("gecme_orani", ascending=False).head(10)


,ders_kodu,kayit_sayisi,ortalama,gecen_sayi,gecme_orani
8,ST201,85,63.906824,61,0.717647
4,DS101,98,65.584898,67,0.683673
5,FN201,91,64.726374,62,0.681319
3,DB201,82,64.048537,55,0.670732
9,WEB101,93,64.059140,62,0.666667
0,AI201,95,61.253684,62,0.652632
7,PY101,91,63.674945,58,0.637363
2,BI201,94,62.412553,56,0.595745
6,MK201,94,59.406809,54,0.574468
1,BA301,77,62.937403,44,0.571429


### 8.3) Sınıf bazında ortalama not


In [ ]:
df.groupby("sinif")["toplam_not"].mean().reset_index().sort_values("sinif")


,sinif,toplam_not
0,1,62.255385
1,2,64.055385
2,3,63.263073
3,4,62.509309


## 9) Pivot table — tablo gibi rapor


Bolum x Kategori bazinda ortalama not


In [ ]:
pivot1 = pd.pivot_table(
    df,
    values="toplam_not",
    index="bolum",
    columns="kategori",
    aggfunc="mean"
)
pivot1


kategori,Business,Data,Math,Tech
bolum,,,,
Bilgisayar,58.263478,72.592000,67.806667,59.224348
Ekonomi,65.640909,61.574667,68.610000,64.809091
Endustri,65.004211,68.508125,56.269333,63.421481
Isletme,64.133973,66.308837,67.746154,62.354769
YBS,60.376790,62.327750,62.364000,63.636032


### 9.1) Pivot table — bolum x ders_kodu


In [ ]:
pivot2 = pd.pivot_table(
    df,
    values="toplam_not",
    index="bolum",
    columns="ders_kodu",
    aggfunc="mean"
)
pivot2


ders_kodu,AI201,BA301,BI201,DB201,DS101,FN201,MK201,PY101,ST201,WEB101
bolum,,,,,,,,,,
Bilgisayar,52.940000,50.450000,57.293333,65.746667,75.525714,64.820000,62.925714,57.453333,67.806667,67.622500
Ekonomi,52.705000,74.160000,69.133333,67.096667,57.893333,52.244000,64.624000,77.086667,68.610000,67.705000
Endustri,67.089412,67.490909,62.331111,69.828000,67.343529,66.906000,65.381111,62.845556,56.269333,60.685263
Isletme,58.184545,64.122500,64.215000,68.194000,64.669565,66.586667,59.324286,66.617000,67.746154,62.637391
YBS,63.052093,61.923889,61.183810,58.970000,65.365714,63.788824,56.264800,62.610909,62.364000,65.436410


## 10) Concat — iki tabloyu alt alta ekleme


Örnek: yeni öğrenci kayıtları geldi diyelim.


In [ ]:
yeni_ogr = pd.DataFrame({
    "ogrenci_id": [999001, 999002],
    "ad": ["Burcu", "Aslı"],
    "soyad": ["Aktaş", "Genç"],
    "sehir": ["Istanbul", "Ankara"],
    "bolum": ["YBS", "Isletme"],
    "sinif": [3, 3],
    "gpa": [3.2, 2.8],
    "email": ["burcu@example.com", "asli@example.com"]
})

ogr_guncel = pd.concat([ogr, yeni_ogr], ignore_index=True)
ogr_guncel.tail()


,ogrenci_id,ad,soyad,sehir,bolum,sinif,gpa,email
217,200218,Selin,Gunes,Antalya,Endustri,3,2.51,selin.gunes93@example.com
218,200219,Can,Erdogan,Istanbul,YBS,3,2.97,can.erdogan52@example.com
219,200220,Selin,Celik,Ankara,YBS,2,1.78,selin.celik63@example.com
220,999001,Burcu,Aktaş,Istanbul,YBS,3,3.20,burcu@example.com
221,999002,Aslı,Genç,Ankara,Isletme,3,2.80,asli@example.com


## 11) Duplicate (tekil olmayan) kontrolu


Örnek: aynı ogrenci_id iki kere gelmiş mi?


In [ ]:
ogr_guncel["ogrenci_id"].duplicated().sum()


np.int64(0)